# Exploration of Working with Youtube Content

⚠️ Warning: This notebook utilizes unofficial Python libraries to access YouTube content. Retrieved content must not be stored or redistributed. All code and tools are intended strictly for educational and research purposes only.

In [ ]:
%pip install smolagents youtube-transcript-api yt-dlp av webvtt-py pysrt torch torchaudio transformers openai
%pip install devtools

from dotenv import load_dotenv
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))
load_dotenv()

## YouTube Video Transcription

### YouTubeTranscriptApi

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from devtools import pprint

ytt_api = YouTubeTranscriptApi()
snippets = ytt_api.fetch("1htKBjuUWec")
pprint(snippets)


In [ ]:
transcript = ""
for s in snippets:
    transcript += f"[{s.start}]\n{s.text}\n[{s.start + s.duration}]\n"
transcript = transcript.strip()
print(transcript)

In [ ]:
from smolagents import tool
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import parse_qs, urlparse


@tool
def transcribe_youtube_vide(url: str) -> str:
    """
    Transcribe a YouTube video.
    Args:
        url (str): The URL of the YouTube video.
    Returns:
        str: The transcript of the video.
    """
    video_id = parse_qs(urlparse(url).query).get("v", [None])[0]
    if not video_id:
        raise ValueError("Invalid YouTube URL.")

    snippets = YouTubeTranscriptApi.fetch(video_id)

    transcript = ""
    for s in snippets:
        transcript += f"[{s.start}]\n{s.text}\n[{s.start + s.duration}]\n"

    return transcript.strip()

## YouTube Video Image Understanding

### Download YouTube Video

In [ ]:
import subprocess
import yt_dlp
import av
import requests
import pysrt
import webvtt
from io import StringIO


def stream_youtube_video(url, capture_interval_sec=5):
    """
    Streams a YouTube video and captures one frame every `capture_interval_sec` seconds.

    Args:
        url (str): YouTube video URL.
        capture_interval_sec (int or float): Time between captured frames in seconds.
    """
    # Get direct video URL from yt-dlp
    ydl_opts = {
        "quiet": True,
        "skip_download": True,
        "format": "bestvideo[ext=mp4][height<=360]+bestaudio[ext=m4a]/best[height<=360]",
        "forceurl": True,
        "noplaylist": True,
        "writesubtitles": True,
        "writeautomaticsub": True,
        "subtitleslangs": ["en"],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)

    title = info.get("title")
    description = info.get("description")
    subtitles = info.get("subtitles", {})
    auto_captions = info.get("automatic_captions", {})
    caption_tracks = subtitles.get("en") or auto_captions.get("en") or []

    structured_captions = []

    srt_track = next((track for track in caption_tracks if track["ext"] == "srt"), None)
    vtt_track = next((track for track in caption_tracks if track["ext"] == "vtt"), None)

    if srt_track:
        response = requests.get(srt_track["url"])
        response.raise_for_status()
        srt_data = response.content.decode("utf-8")

        def to_sec(t):
            return t.hours * 3600 + t.minutes * 60 + t.seconds + t.milliseconds / 1000

        structured_captions = [
            {
                "start": to_sec(sub.start),
                "end": to_sec(sub.end),
                "text": sub.text.strip(),
            }
            for sub in pysrt.from_str(srt_data)
        ]
    if vtt_track:
        response = requests.get(vtt_track["url"])
        response.raise_for_status()
        vtt_data = response.text

        vtt_file = StringIO(vtt_data)

        def to_sec(t):
            """Convert 'HH:MM:SS.mmm' to float seconds"""
            h, m, s = t.split(":")
            s, ms = s.split(".")
            return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000

        for caption in webvtt.read_buffer(vtt_file):
            structured_captions.append(
                {
                    "start": to_sec(caption.start),
                    "end": to_sec(caption.end),
                    "text": caption.text.strip(),
                }
            )

    video_format = next(
        f
        for f in info["formats"]
        if f.get("vcodec") != "none" and f.get("height") == 360
    )
    video_url = video_format["url"]

    # Use ffmpeg to stream video to stdout
    ffmpeg_cmd = [
        "ffmpeg",
        "-i",
        video_url,
        "-f",
        "matroska",  # container format
        "-",
    ]

    process = subprocess.Popen(
        ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL
    )

    container = av.open(process.stdout)
    stream = container.streams.video[0]
    time_base = stream.time_base

    next_capture_time = 0

    imgs = []
    for frame in container.decode(stream):
        if frame.pts is None:
            continue

        timestamp = float(frame.pts * time_base)
        if timestamp >= next_capture_time:
            img = frame.to_image()  # PIL image
            print(f"IMG: {len(imgs)}")
            display(img)
            imgs.append(img)
            next_capture_time += capture_interval_sec

    process.terminate()

    return {
        "title": title,
        "description": description,
        "captions": structured_captions,
        "frames": imgs,
    }


video = stream_youtube_video(
    # "https://www.youtube.com/watch?v=1htKBjuUWec",
    "https://www.youtube.com/watch?v=L1vXCYZAYYM",
    capture_interval_sec=2,
)

print(video["title"])
print(video["description"])
for caption in video["captions"]:
    print(f"{caption['start']} - {caption['end']}:\n{caption['text']}")
imgs = video["frames"]

### Zero-shot Object Detection and Grounding

Setting up:

In [ ]:
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, pipeline

checkpoint = "google/owlv2-base-patch16-ensemble"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

model = AutoModelForZeroShotObjectDetection.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

image = imgs[41].copy()

Processing:

In [ ]:
import torch
from PIL import ImageDraw
from collections import Counter


def process_with_owl(image, text_queries):
    inputs = processor(text=text_queries, images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        target_sizes = torch.tensor([image.size[::-1]])
        results = processor.post_process_object_detection(
            outputs, threshold=0.1, target_sizes=target_sizes
        )[0]

    # Display results

    image_labeled = imgs[41].copy()
    draw = ImageDraw.Draw(image_labeled)

    scores = results["scores"].tolist()
    labels = results["labels"].tolist()
    boxes = results["boxes"].tolist()

    for box, score, label in zip(boxes, scores, labels):
        xmin, ymin, xmax, ymax = box
        draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
        draw.text(
            (xmin, ymin), f"{text_queries[label]}: {round(score, 2)}", fill="white"
        )

    print(f"Text queries: {text_queries}")
    print(f"Labels: {Counter(labels)}")
    display(image_labeled)


process_with_owl(image, ["emperor penguin", "adelie penguin", "petrel"])
process_with_owl(image, ["penguin", "petrel"])
process_with_owl(image, ["bird"])
process_with_owl(image, ["bird species"])


Challenges:
- require predefined and crafted text queries (labels)
- can label same object twice with different probability
- hard to choose proper threshold for open questions

### Image Captioning and Visual Question Answering

#### BLIP2

Setting up:

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

# model_id = "Salesforce/blip2-opt-2.7b"
model_id = "Salesforce/blip2-flan-t5-xl"
# model_id = "Salesforce/blip2-opt-2.7b-coco"

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
processor = Blip2Processor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id)
model.eval().to(device)

image = imgs[41].copy().convert("RGB")

Processing:

In [ ]:
import torch


def process_with_blip2(case, image, text=None):
    inputs = processor(image, text=text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    if device.type == "mps":
        inputs = {
            k: v.to(torch.float32) if v.dtype.is_floating_point else v
            for k, v in inputs.items()
        }

    with torch.no_grad():
        out = model.generate(**inputs)
        # out = model.generate(**inputs, min_new_tokens=50, max_new_tokens=100)

    print(f"{case}: {text}|{processor.decode(out[0], skip_special_tokens=True)}")


display(image)
process_with_blip2("Captioning", image)
process_with_blip2("Completion", image, "Here are birds of species ")
process_with_blip2("VQA", image, "What bird species are here?")
process_with_blip2("VQA", image, "What birds are here?")

#### BLIP

Setting Up:

In [ ]:
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    BlipForQuestionAnswering,
)
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

blip_id = "Salesforce/blip-image-captioning-base"
# blip_id = "Salesforce/blip-image-captioning-large"
blip_vqa_id = "Salesforce/blip-vqa-base"
# blip_vqa_id = "Salesforce/blip-vqa-capfilt-large"

blip_processor = BlipProcessor.from_pretrained(blip_id)
blip_model = BlipForConditionalGeneration.from_pretrained(blip_id).to(device)
blip_vqa_model = BlipForQuestionAnswering.from_pretrained(blip_vqa_id).to(device)

image = imgs[41].copy().convert("RGB")

Processing:

In [ ]:
display(image)


def process_with_blip(model, case, image, text=None):
    blip_inputs = blip_processor(images=image, text=text, return_tensors="pt")

    blip_inputs = {k: v.to(device) for k, v in blip_inputs.items()}
    if device.type == "mps":
        blip_inputs = {
            k: v.to(torch.float32) if v.dtype.is_floating_point else v
            for k, v in blip_inputs.items()
        }

        try:
            generated_ids = model.generate(**blip_inputs)
            # generated_ids = model.generate(**blip_inputs, min_new_tokens=250, max_new_tokens=1000)
        except TypeError:
            print((f"{case}: FAILED!"))
            return

        response = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
        print(f"{case}: {text}|{'\n'.join(response)}")
        return response


process_with_blip(blip_model, "BLIP Caption", image)
process_with_blip(blip_model, "BLIP Completion", image, "Birds species here are ")
process_with_blip(
    blip_model,
    "BLIP Question Answering",
    image,
    "What birds are here?",
)
process_with_blip(blip_vqa_model, "BLIP VQA Caption", image)
process_with_blip(
    blip_vqa_model, "BLIP VQA Completion", image, "Birds species here are "
)
process_with_blip(
    blip_vqa_model,
    "BLIP VQA Question Answering",
    image,
    "What birds are here?",
);

Batch question processing:

In [ ]:
blip_inputs = blip_processor(
    images=[img.copy().convert("RGB") for img in imgs],
    text=["What birds are here?"] * len(imgs),
    return_tensors="pt",
)
blip_inputs = {k: v.to(device) for k, v in blip_inputs.items()}
if device.type == "mps":
    blip_inputs = {
        k: v.to(torch.float32) if v.dtype.is_floating_point else v
        for k, v in blip_inputs.items()
    }
generated_ids = blip_vqa_model.generate(**blip_inputs, max_new_tokens=50)

labels = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
answer = "\n".join(labels)
print(f"BLIP VQA Answer:\n{answer}")

labels = list(set(labels))

### Objects Grounding

Setting up:

In [ ]:
from transformers import (
    AutoProcessor,
    AutoModelForZeroShotObjectDetection,
)
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

dino_id = "IDEA-Research/grounding-dino-base"
dino_processor = AutoProcessor.from_pretrained(dino_id)
dino_model = AutoModelForZeroShotObjectDetection.from_pretrained(dino_id).to(device)

image = imgs[41].copy().convert("RGB")

Processing:

In [ ]:
from PIL import ImageDraw
import torch
from collections import Counter


def process_with_dino(image, prompt=None):
    print(f"Prompt: {prompt}")
    inputs = dino_processor(text=prompt, images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino_model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]]).to(device)
    results = dino_processor.post_process_grounded_object_detection(
        outputs, target_sizes=target_sizes, threshold=0.3
    )[0]

    # Display the results

    image_labeled = image.copy()
    draw = ImageDraw.Draw(image_labeled)

    scores = results["scores"].tolist()
    detected_labels = results["text_labels"]
    boxes = results["boxes"]

    print(f"Objects Detected: {len(boxes)}")
    print(f"Labels: {Counter(detected_labels)}")
    for box, score, label in zip(boxes, scores, detected_labels):
        xmin, ymin, xmax, ymax = box
        draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
        draw.text((xmin + 2, ymin), f"{label}: {round(score, 2)}", fill="red")

    display(image_labeled)
    return boxes


boxes = process_with_dino(image, "bird")
boxes = process_with_dino(image, "bird.")
try:
    boxes = process_with_dino(image, ". ".join(labels) + ".")
except NameError:
    pass  # BLIP VQA Answer not available
boxes = process_with_dino(image, "Emperor Penguin. Adelie Penguins. Giant Petrel.")
boxes = process_with_dino(image, "emperor penguin. adelie penguins. giant petrel.")
boxes = process_with_dino(image, "penguin. petrel.")


### Objects Labeling

Setting up:

In [ ]:
from transformers import (
    CLIPProcessor,
    CLIPModel,
)
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

clip_id = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(clip_id)
clip_model = CLIPModel.from_pretrained(clip_id).to(device)

image = imgs[41].copy().convert("RGB")

Processing:

In [ ]:
from collections import Counter


def process_with_clip(image, labels):
    labeled_boxes = []

    for box in boxes:
        xmin, ymin, xmax, ymax = map(int, box.tolist())
        cropped = image.crop((xmin, ymin, xmax, ymax))

        clip_inputs = clip_processor(
            text=labels, images=cropped, return_tensors="pt", padding=True
        ).to(device)

        with torch.no_grad():
            outputs = clip_model(**clip_inputs)
            logits = outputs.logits_per_image[0]
            probs = logits.softmax(dim=0)
            best_idx = probs.argmax().item()
            label = labels[best_idx]
            score = probs[best_idx].item()
            labeled_boxes.append((box, label, score))

    # Display the results

    image_labeled = image.copy()
    draw = ImageDraw.Draw(image_labeled)

    labels_detected = []
    for box, label, score in labeled_boxes:
        labels_detected.append(label)
        xmin, ymin, xmax, ymax = map(int, box.tolist())
        draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
        draw.text((xmin, ymin), f"{label}: {score:.2f}", fill="white")

    print(f"Labels: {Counter(labels_detected)}")
    display(image_labeled)


print("Actual Labels:")
process_with_clip(image, ["adelie penguin", "emperor penguin", "giant petrel"])
print("Generalized Labels:")
process_with_clip(image, ["penguin", "petrel"])
try:
    print("BLIP VQA Labels:")
    boxes = process_with_clip(image, labels)
except NameError:
    pass  # BLIP VQA Answer not available

## Video Comprehension with OpenAI

In [ ]:
from openai import OpenAI
from io import BytesIO
import base64

client = OpenAI()

base64_frames = []
for img in imgs:
    buffered = BytesIO()
    img.save(buffered, format="JPEG")
    encoded = base64.b64encode(buffered.getvalue()).decode("utf-8")
    base64_frames.append(encoded)


def process_with_openai(frames_subset):
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": (
                            """\
    These are some frames from a video that I want to upload.

    Video description:
    Emperor Penguin Chicks and Adelie Penguins stand up to Giant Petrel
    The emperor penguin chicks are left to fend for themselves and stand up against a giant petrel with the help of a feisty Adelie.
    From our programme 'Spy in the Snow' for the BBC.

    Name all bird species to be on camera simultaneously.
    """
                        ),
                    },
                    *[
                        {
                            "type": "input_image",
                            "image_url": f"data:image/jpeg;base64,{frame}",
                        }
                        for frame in frames_subset
                    ],
                ],
            }
        ],
    )

    print(response.output_text)


print("Expected: only 2 bird species are visible at the same time.")
print("Model Response: ")
process_with_openai(base64_frames[30:39])

print("\n\n")

print("Expected: all 3 bird species are visible at the same time.")
print("Model Response: ")
process_with_openai(base64_frames[30:45])

## YouTube Tool Implementation

In [ ]:
from smolagents import Tool
from openai import OpenAI
from tools import SpeechRecognitionTool
from io import BytesIO
import yt_dlp
import av
import torchaudio
import subprocess
import requests
import base64
from devtools import pprint


class YoutubeVideoTool(Tool):
    name = "youtube_video"
    description = """Process the video and return the requested information from it."""
    inputs = {
        "url": {
            "type": "string",
            "description": "The URL of the YouTube video.",
        },
        "query": {
            "type": "string",
            "description": "The question to answer.",
        },
    }
    output_type = "string"

    def __init__(
        self,
        video_quality: int = 360,
        frames_interval: int | float | None = 2,
        chunk_duration: int | float | None = 20,
        speech_recognition_tool: SpeechRecognitionTool | None = None,
        client: OpenAI | None = None,
        model_id: str = "gpt-4.1-mini",
        debug: bool = False,
        **kwargs,
    ):
        self.video_quality = video_quality
        self.speech_recognition_tool = speech_recognition_tool
        self.frames_interval = frames_interval
        self.chunk_duration = chunk_duration

        self.client = client or OpenAI()
        self.model_id = model_id

        self.debug = debug

        super().__init__(**kwargs)

    def forward(self, url: str, query: str):
        """
        Process the video and return the requested information.
        Args:
            url (str): The URL of the YouTube video.
            query (str): The question to answer.
        Returns:
            str: Answer to the query.
        """
        answer = ""
        for chunk in self._split_video_into_chunks(url):
            prompt = self._prompt(
                chunk,
                query,
                answer,
            )
            response = client.responses.create(
                model="gpt-4.1-mini",
                input=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "input_text",
                                "text": prompt,
                            },
                            *[
                                {
                                    "type": "input_image",
                                    "image_url": f"data:image/jpeg;base64,{frame}",
                                }
                                for frame in self._base64_frames(chunk["frames"])
                            ],
                        ],
                    }
                ],
            )
            answer = response.output_text
            if self.debug:
                print(
                    f"CHUNK {chunk['start']} - {chunk['end']}:\n\n{prompt}\n\nANSWER:\n{answer}"
                )

        return answer

    def _prompt(self, chunk, query, aggregated_answer):
        prompt = [
            f"""\
These are some frames of a video that I want to upload.
I will ask a question about the entire video, but I will only last part of it.

VIDEO TITLE:
{chunk["title"]}

VIDEO DESCRIPTION:
{chunk["description"]}

FRAMES SUBTITLES:
{chunk["captions"]}"""
        ]

        if aggregated_answer:
            prompt.append(f"""\
Here is the answer to the same question based on the previous video parts:
                          
BASED ON PREVIOUS PARTS:
{aggregated_answer}""")

        prompt.append(f"""\
Now, please answer the question.
                      
QUESTION:
{query}""")

        return "\n\n".join(prompt)

    def _split_video_into_chunks(
        self, url: str, with_captions: bool = True, with_frames: bool = True
    ):
        video = self._process_video(
            url, with_captions=with_captions, with_frames=with_frames
        )
        video_duration = video["duration"]
        chunk_duration = self.chunk_duration or video_duration

        chunk_start = 0.0
        while chunk_start < video_duration:
            chunk_end = min(chunk_start + chunk_duration, video_duration)
            chunk = self._get_video_chunk(video, chunk_start, chunk_end)
            yield chunk
            chunk_start += chunk_duration

    def _get_video_chunk(self, video, start, end):
        chunk_captions = [
            c for c in video["captions"] if c["start"] <= end and c["end"] >= start
        ]
        chunk_frames = [
            f
            for f in video["frames"]
            if f["timestamp"] >= start and f["timestamp"] <= end
        ]

        return {
            "title": video["title"],
            "description": video["description"],
            "start": start,
            "end": end,
            "captions": "\n".join([c["text"] for c in chunk_captions]),
            "frames": chunk_frames,
        }

    def _process_video(
        self, url: str, with_captions: bool = True, with_frames: bool = True
    ):
        lang = "en"
        info = self._get_video_info(url, lang)

        if with_captions:
            captions = self._extract_captions(
                lang, info.get("subtitles", {}), info.get("automatic_captions", {})
            )
            if not captions and self.speech_recognition_tool:
                audio_url = self._select_audio_format(info["formats"])
                audio = self._capture_audio(audio_url)
                waveform, sample_rate = torchaudio.load(audio)
                assert sample_rate == 16000
                waveform_np = waveform.squeeze().numpy()
                captions = self.speech_recognition_tool.transcribe(waveform_np)
        else:
            captions = []

        if with_frames:
            video_url = self._select_video_format(info["formats"], 360)["url"]
            frames = self._capture_video_frames(video_url, self.frames_interval)
        else:
            frames = []

        return {
            "id": info["id"],
            "title": info["title"],
            "description": info["description"],
            "duration": info["duration"],
            "captions": captions,
            "frames": frames,
        }

    def _get_video_info(self, url: str, lang: str):
        ydl_opts = {
            "quiet": True,
            "skip_download": True,
            "format": "bestvideo[ext=mp4][height<=360]+bestaudio[ext=m4a]/best[height<=360]",
            "forceurl": True,
            "noplaylist": True,
            "writesubtitles": True,
            "writeautomaticsub": True,
            "subtitlesformat": "vtt",
            "subtitleslangs": [lang],
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)

        return info

    def _extract_captions(self, lang, subtitles, auto_captions):
        caption_tracks = subtitles.get(lang) or auto_captions.get(lang) or []

        structured_captions = []

        srt_track = next(
            (track for track in caption_tracks if track["ext"] == "srt"), None
        )
        vtt_track = next(
            (track for track in caption_tracks if track["ext"] == "vtt"), None
        )

        if srt_track:
            import pysrt

            response = requests.get(srt_track["url"])
            response.raise_for_status()
            srt_data = response.content.decode("utf-8")

            def to_sec(t):
                return (
                    t.hours * 3600 + t.minutes * 60 + t.seconds + t.milliseconds / 1000
                )

            structured_captions = [
                {
                    "start": to_sec(sub.start),
                    "end": to_sec(sub.end),
                    "text": sub.text.strip(),
                }
                for sub in pysrt.from_str(srt_data)
            ]
        if vtt_track:
            import webvtt
            from io import StringIO

            response = requests.get(vtt_track["url"])
            response.raise_for_status()
            vtt_data = response.text

            vtt_file = StringIO(vtt_data)

            def to_sec(t):
                """Convert 'HH:MM:SS.mmm' to float seconds"""
                h, m, s = t.split(":")
                s, ms = s.split(".")
                return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000

            for caption in webvtt.read_buffer(vtt_file):
                structured_captions.append(
                    {
                        "start": to_sec(caption.start),
                        "end": to_sec(caption.end),
                        "text": caption.text.strip(),
                    }
                )
        return structured_captions

    def _select_video_format(self, formats, video_quality):
        video_format = next(
            f
            for f in formats
            if f.get("vcodec") != "none" and f.get("height") == video_quality
        )
        return video_format

    def _capture_video_frames(self, video_url, capture_interval_sec=None):
        ffmpeg_cmd = [
            "ffmpeg",
            "-i",
            video_url,
            "-f",
            "matroska",  # container format
            "-",
        ]

        process = subprocess.Popen(
            ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL
        )

        container = av.open(process.stdout)
        stream = container.streams.video[0]
        time_base = stream.time_base

        frames = []
        next_capture_time = 0
        for frame in container.decode(stream):
            if frame.pts is None:
                continue

            timestamp = float(frame.pts * time_base)
            if capture_interval_sec is None or timestamp >= next_capture_time:
                frames.append(
                    {
                        "timestamp": timestamp,
                        "image": frame.to_image(),  # PIL image
                    }
                )
                if capture_interval_sec is not None:
                    next_capture_time += capture_interval_sec

        process.terminate()
        return frames

    def _base64_frames(self, frames):
        base64_frames = []
        for f in frames:
            buffered = BytesIO()
            f["image"].save(buffered, format="JPEG")
            encoded = base64.b64encode(buffered.getvalue()).decode("utf-8")
            base64_frames.append(encoded)
        return base64_frames

    def _select_audio_format(self, formats):
        audio_formats = [
            f
            for f in formats
            if f.get("vcodec") == "none"
            and f.get("acodec")
            and f.get("acodec") != "none"
        ]

        if not audio_formats:
            raise ValueError("No valid audio-only formats found.")

        # Prefer m4a > webm, highest abr first
        preferred_exts = ["m4a", "webm"]

        def sort_key(f):
            ext_score = (
                preferred_exts.index(f["ext"]) if f["ext"] in preferred_exts else 99
            )
            abr = f.get("abr") or 0
            return (ext_score, -abr)

        audio_formats.sort(key=sort_key)
        return audio_formats[0]["url"]

    def _capture_audio(self, audio_url) -> BytesIO:
        audio_buffer = BytesIO()
        ffmpeg_audio_cmd = [
            "ffmpeg",
            "-i",
            audio_url,
            "-f",
            "wav",
            "-acodec",
            "pcm_s16le",  # Whisper prefers PCM
            "-ac",
            "1",  # Mono
            "-ar",
            "16000",  # 16kHz for Whisper
            "-",
        ]

        result = subprocess.run(
            ffmpeg_audio_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        if result.returncode != 0:
            raise RuntimeError("ffmpeg failed:\n" + result.stderr.decode())

        audio_buffer = BytesIO(result.stdout)
        audio_buffer.seek(0)
        return audio_buffer

Verify the tool:

In [ ]:
client = OpenAI()
speech_recognition_tool = SpeechRecognitionTool()
youtube_tool = YoutubeVideoTool(
    client=client,
    speech_recognition_tool=speech_recognition_tool,
    frames_interval=3,
    chunk_duration=60,
    debug=True,
)

r = youtube_tool(
    # url="https://www.youtube.com/watch?v=1htKBjuUWec",
    # query="What does Teal'c say in response to the question \"Isn't that hot?\"?",
    url="https://www.youtube.com/watch?v=L1vXCYZAYYM",
    query="what is the highest number of bird species to be on camera simultaneously?",
)
print(f"\n\nFINAL ANSWER:\n{r}")

## Agent with YouTube Tool

In [ ]:
from smolagents import ToolCallingAgent, OpenAIServerModel

model = model = OpenAIServerModel(model_id="gpt-4.1")

speech_recognition_tool = SpeechRecognitionTool()
youtube_tool = YoutubeVideoTool(
    client=model.client,
    speech_recognition_tool=speech_recognition_tool,
    frames_interval=3,
    chunk_duration=60,
)
agent = ToolCallingAgent(
    model=model,
    tools=[youtube_tool],
)

for t in [
    "In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?",
    "Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec. What does Teal'c say in response to the question \"Isn't that hot?\"",
]:
    r = agent.run(t)